## 1. Welfare in the Eaton and Kortum (2002) economy
Consider a three country world, where the endowment of labor in each country is 1. There is a continuum of goods $k$ and the technology for producing a good $k$ in country $i$ is $y_i(k) = z_i(k)\ell_i(k)$. Preferences are of constant elasticity of substitution form, with the elasticity set to 2. Assume that the distributions over $z$ are Frechet with  $\theta = 4$.

## Solving the model
The code below solves the N-country model. First, I set up some useful functions, then I solve the models. The general algorithm is 

0. Draw productivity values for each country (1...N) and good (1...J): NxJ total productivities.
1. Guess a vector of wages (country 1 has the normalization w=1)
2. Given the wage vector, find the prices each country offers for each good-destination
3. Choose the lowest cost provider for each good to each country
4. Use the demand function to compute expenditure shares and the aggregate trade balance for each country
5. Is trade balanced? If so, we have an equilibrium. If not, adjust the guess of wages and return to step 2.

I will solve this by writing a function that takes w as an input and returns the vector of trade balances. Then, I will pass this funciton to a nonlinear solver and have it find the w that zeros the function. 

\[I am writing this in python because I find it easy to read. I am also using this as practice at writing better python. This is a pretty easy model to solve, but python is not (out of the box) the best language for solving hard numerical problems.\]

In [1]:
import numpy as np      # Numerical computing in python
import scipy.optimize   # Root finding routines

# Format output so floats have 3 decimal places
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

Set up the parameters. 

In [2]:
N = 3                       # Number of countries
J = 200000                 # Number of goods (this needs to be a big number)
theta = 4                   # Frechet spread parameter
T = np.ones((N,1))*1.5      # Frechet location parameters
rho = 0.5                   # 1/(1-rho) is the elasticity of substitution
tau = np.ones((N,N))        # Trade costs.  (1+tau) in the notes
L = np.ones((N,1))          # Country sizes

rand_seed = 4               # Seed for the random number generator (for reproducibility)

Define the inverse Frechet cdf function. To draw random Frechet values, we draw uniform (0,1) numbers and pass them through the inverse Frechet. 

In [3]:
def inverse_Frechet(u,T,theta):
    return (np.log(u)/-T )**(-1/theta)

This function draws the J-by-N matrix of marginal costs, z.

In [4]:
def draw_mc(J, N, T, theta, rand_seed):
    
    np.random.seed(seed = rand_seed)                  # Set random generator seed for reproducability
    z = np.random.uniform(size=(J,N))                 # Draw uniform random numbers

    # Apply the inverse Frechet 
    for n in range(0,N):                       
        z[:,n] = inverse_Frechet(z[:,n], T[n], theta)
    
    return z

This function takes the wages and productivity draws and computes the trade shares for each country

In [5]:
def compute_shares(wage, z, tau, N, J, rho):
    
    # Add the normalized home country wage to the wage vector 
    w = np.insert(wage, 0, 1)
    
    # allocate the return variables
    shares = np.zeros((N,N)) 
    P = np.empty((N,1))
    
    for n in range(0, N):
        # find the country that provides n with lowest cost of each good
        i = np.argmin(w*tau[n, :]/z, axis = 1)  # Numpy broadcasts the vector tau[n,:] across the rows of z
                
        # compute prices, price index, and shares
        p = (w*tau[n, :]/z)[np.arange(J), i]                # values associated with the argmin above
        
        P[n, 0] = ( np.sum(p**(rho/(rho-1))) * 1/J ) ** ((rho-1)/rho)
        s = 1/J * (p / P[n, 0]) ** (-rho/(1-rho) )
        
        # Is there a better way to do this?
        for ex in range(0,N):
            shares[n, ex] = np.sum( s[i==ex] )
    
    
    return shares, P
    
    

In [6]:
def trade_balance(wage, L, z, tau, N, J, rho, opt):
    
    # For the given wage vector and parameters, compute the agg. trade shares
    shares, P = compute_shares(wage, z, tau, N, J, rho)
    
    # Compute the trade balance.  Do this by computing the trade balance with each country and suming them up.
    w = np.insert(wage, 0, 1)    
    inc = np.multiply(w, np.transpose(L))                     # income in country n is wage times labor
    
    tb = np.zeros((N,1))
    
    for n in range(0,N):
        for i in range(0,N):
            if i != n:
                tb[n,0]  = tb[n,0] + shares[i,n]*inc[0,i] - shares[n,i]*inc[0,n]
    
    # If I am using this funtion in a optimization routine, do not return the first trade balance. It is zero by Walras Law.
    if opt == 1:
        return tb[1:,0]
    else:
        return tb


Once we have solved the model, this function reports useful equilibrium objects. 

In [7]:
def report_results(sol, z, tau, N, J, rho):
    
    w = np.insert(sol, 0, 1)
    inc = np.multiply(w, np.transpose(L))
    s, P = compute_shares(sol, z, tau, N, J, rho)
    tb = trade_balance(sol, L, z, tau, N, J, rho, 0)
    welfare = inc/np.transpose(P)

    print('The equilibrium wage vector is:', w)
    print('\nThe trade share matrix is:\n', s)
    print('\nThe trade balances are:\n', tb)
    print('\nWelfare (wL/P) is:', welfare)

**a. Simple and symmetric.** Let there be no trade costs, i.e., $\tau_{ni} = 0$, and let $T_i = 1.5$ for all $i$. Report the equilibrium bilateral trade share matrix. (An element of the matrix is $\pi_{ ni}$, the share of total spending in $n$ on goods from $i$.) The solution to this model is trivial, so this is a good place to first check that our programs are working.

In [8]:
# Draw marginal costs
z = draw_mc(J, N, T, theta, rand_seed)

# Coutnries are symmetric, so all have w = 1 in equilibrium
# I am not solving for the equilbrium here, just computing the expenditure shares

w = np.ones((2,1))
s, P = compute_shares(w, z, tau, N, J, rho)


print('The expenditure matrix is:')
print(s)

The expenditure matrix is:
[[0.335 0.332 0.333]
 [0.335 0.332 0.333]
 [0.335 0.332 0.333]]


Increasing J makes this closer to a matrix of 1/3. Larger J slows down solving for equilibrium below. 

**b. Symmetric geography.** Now introduce iceberg trade costs. Let $\tau_{ ni} = 0.1$ for each $n \neq i$,
and keep the remaining parameters as in part (a). Report the bilateral trade share matrix.

In [9]:
# Draw marginal costs
z = draw_mc(J, N, T, theta, rand_seed)

# Create the trade cost matrix
tau = np.ones((N,N))*1.1
for n in range(0,N):
    tau[n, n] = 1.0

print('The trade cost matrix is now:\n', tau)


The trade cost matrix is now:
 [[1.000 1.100 1.100]
 [1.100 1.000 1.100]
 [1.100 1.100 1.000]]


In [10]:
# Countries are still symmetric, so all have w = 1 in equilibrium

w = np.ones((2,1))
s, P = compute_shares(w, z, tau, N, J, rho)

print('\nThe expenditure matrix is:')
print(s)


The expenditure matrix is:
[[0.424 0.288 0.289]
 [0.290 0.422 0.289]
 [0.291 0.288 0.422]]


**c. Asymmetric geography.** Countries have identical technologies, $T_i = 1.5$ for all $i$, and $\theta = 4$. Country 3, however, is 'far away' from countries 1 and 2: $\tau_{12} = \tau_{ 21} = 1.05$ and $\tau_{ 13} = \tau_{ 31} = \tau_{ 32} = \tau_{ 23} = 1.3$. Report the equilibrium bilateral trade share matrix. (An element of the matrix is $\pi_{ni}$, the share of total spending in $n$ on goods from $i$.) Report an index of welfare in each country, $w_i /P_i$ , where $P_i$ is the CES aggregate price index. \[My equilibrium wage vector is: (1, 1, 0.966).\]

In [11]:

tau[0,:] = [1.0, 1.05, 1.3]
tau[1,:] = [1.05, 1.0, 1.3]
tau[2,:] = [1.3, 1.3, 1.0]
print('The trade cost matrix is:\n', tau)

# Draw marginal costs
z = draw_mc(J, N, T, theta, rand_seed)


The trade cost matrix is:
 [[1.000 1.050 1.300]
 [1.050 1.000 1.300]
 [1.300 1.300 1.000]]


The world is no longer symmetric. Now we have to solve for the equilibrium wage vector. To do so, we need to choose $w_2$ and $w_3$ so that the aggregate trade balance for countries 2 and 3 are zero. We normalized $w_1=1$ and we get balanced trade in country 1 by Walras Law.  

In [12]:
# Solve model
wage0 = np.ones((2,1))
sol  = scipy.optimize.newton_krylov(lambda k: trade_balance(k, L, z, tau, N, J, rho, 1), wage0, f_tol = 0.0001)

In [13]:
# Report results
report_results(sol, z, tau, N, J, rho)

The equilibrium wage vector is: [1.000 0.998 0.963]

The trade share matrix is:
 [[0.449 0.369 0.182]
 [0.369 0.448 0.182]
 [0.190 0.188 0.622]]

The trade balances are:
 [[0.000]
 [-0.000]
 [0.000]]

Welfare (wL/P) is: [[1.660 1.657 1.527]]


**d. Technological progress.** Let $T_2 = 3$, and keep the remaining parameters as in part
(c). This is a technological improvement in country 2. \[You will need to redraw your
productivities.\] Report the bilateral trade share matrix. 

Compare welfare in this economy with welfare in the economy in part (c). Discuss your findings in the context of how an
increase in technology in one country benefits other countries. \[My equilibrium wage vector
is: (1, 1.14, 0.962).\]

In [14]:
# Set parameters
T[1] = 3

# Draw marginal costs
z = draw_mc(J, N, T, theta, rand_seed)

tau[0,:] = [1.0, 1.05, 1.3]
tau[1,:] = [1.05, 1.0, 1.3]
tau[2,:] = [1.3, 1.3, 1.0]
print('The trade cost matrix is:\n', tau)


The trade cost matrix is:
 [[1.000 1.050 1.300]
 [1.050 1.000 1.300]
 [1.300 1.300 1.000]]


In [15]:
# Solve the model
wage0 = np.ones((2,1))
sol  = scipy.optimize.newton_krylov(lambda k: trade_balance(k, L, z, tau, N, J, rho, 1), wage0, f_tol = 0.0001)

In [16]:
# Report results
report_results(sol, z, tau, N, J, rho)


The equilibrium wage vector is: [1.000 1.149 0.960]

The trade share matrix is:
 [[0.426 0.398 0.176]
 [0.347 0.480 0.174]
 [0.183 0.208 0.609]]

The trade balances are:
 [[0.000]
 [-0.000]
 [-0.000]]

Welfare (wL/P) is: [[1.682 1.938 1.535]]


**e. Frechet dispersion.** Now change $\theta = 8$, and keep the remaining parameters as in part (c)
(i.e., change $T_2 = 1.5$). \[You will need to redraw your productivities.\] Report the bilateral
trade share matrix. Compare welfare in this economy with welfare in the economy in part
(c). What is the intuition for this result? How does the change in $\theta$ affect countries 1 and
2 compared to country 3? Why? \[My equilibrium wage vector is: (1, 1, 0.978).\]

In [17]:
# Set up parameters
theta = 8
T[1] = 1.5

tau[0,:] = [1.0, 1.05, 1.3]
tau[1,:] = [1.05, 1.0, 1.3]
tau[2,:] = [1.3, 1.3, 1.0]
print('The trade cost matrix is:\n', tau)

# Draw the marginal costs
z = draw_mc(J, N, T, theta, rand_seed)

The trade cost matrix is:
 [[1.000 1.050 1.300]
 [1.050 1.000 1.300]
 [1.300 1.300 1.000]]


In [18]:
# Solve model
wage0 = np.ones((2,1))
sol  = scipy.optimize.newton_krylov(lambda k: trade_balance(k, L, z, tau, N, J, rho, 1), wage0, f_tol = 0.0001)

In [19]:
# Report results
report_results(sol, z, tau, N, J, rho)


The equilibrium wage vector is: [1.000 0.999 0.975]

The trade share matrix is:
 [[0.547 0.371 0.082]
 [0.371 0.548 0.082]
 [0.085 0.083 0.832]]

The trade balances are:
 [[0.000]
 [-0.000]
 [0.000]]

Welfare (wL/P) is: [[1.237 1.236 1.173]]
